In [1]:
import re
from collections import Counter
import pandas as pd 
import numpy as np




import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')





import difflib



import math
WORD = re.compile(r"\w+") 




In [2]:
def words(text): return re.findall(r'\w+', text.lower())

In [3]:
data = pd.read_csv("D:/Indian Personality/first10100.csv") 

In [4]:
data_categories = data['Categories']
title = data['Title']
content = data['Content']

In [5]:
"""
Now we'll convert data_categories into proper list to be used 
"""

def to_list(s) :
    s = s.replace("[","")
    s = s.replace("]","")
    s = s.replace("'","")
    s = s.replace("'","")
    
    return list(s.split(","))

In [6]:
%%time 


for i in range(10088) :
    data_categories[i] = to_list(data_categories[i])

<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 1.03 s


In [7]:
import re

# If you need to use the regex more than once it is suggested to compile it.
pattern = re.compile(r"[\n]+")
pattern2 = re.compile(r"[=]+")

In [8]:
%%time
for i in range(10088) :
    content[i] = pattern.sub("",content[i])
    content[i] = pattern2.sub("",content[i])

<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 4.63 s


In [9]:
%%time

text = " "
for i in range(10088) :
    text += ''.join(data_categories[i])
    
    
for i in range(10088) :
    text += title[i]
    text += " "
    


Wall time: 6.81 s


In [10]:
%%time
WORDS = Counter(words(text))

Wall time: 369 ms


### 1st Imp function( i.e Word correction function) .

###### It will correct spelling of any query and aprt from that it'll also return you some suggestion which our system thinks you may have wanted to type

In [11]:
def correction_inside(word) :
    words = word.split()
    final_suggestion_list = []
    final_suggestion_dict = {}
    final_ans = []
    for w in words :
        w = w.lower()
        cor_word = []
        counter = 0 
        max_score = 0 
        text = " "
        for i in WORDS :
            i = i.lower()
            #print(i,word)
            #print(difflib.SequenceMatcher(None,word,i).ratio()) 
            score = difflib.SequenceMatcher(None,w,i).ratio()
            """
            Apart from sending max word we'll also certain other combinations 
            """
                        
            if(score > 0.7) :
                cor_word.append(i)
                
            
            
            if(score > max_score) :
                max_score = score 
                text = i
        final_ans.append(text)
        final_suggestion_dict[w] = cor_word
                
            

        
            
            
    return final_ans , final_suggestion_dict

In [12]:
def correction(words) :
    words = words.split(" ")
    word = "" 
    for w in words :
        w = w.lower()
        if w not in stop :
            word += w
            word += " "
    #print("Input query  ",word)
    correct_word , suggestions = correction_inside(word) 
    print("Searching for .......    ",correct_word,"\n")
    print("Here is the list of similar words you may be looking for ...............\n")
    for k in suggestions :
        print(k,"    ",suggestions[k],"\n")
    
    
    print("If the suggestion is correct then press continue otherwise you can research your query")
    return correct_word

In [13]:
%%time

### Keyword will store all keywords 

keyword = [0 for i in range(10000000)]
counter = 0
for i in range(10088) :
    keyword[counter] = title[i].lower()
    counter += 1 
    
for i in range(10088) :
    for j in range(len(data_categories[i])) :
        keyword[counter] = data_categories[i][j].lower()
        counter += 1 


Wall time: 3.15 s


In [14]:
keyword = keyword[:counter]
keyword = list(set(keyword))
counter = len(keyword)
print(counter)

37199


In [47]:
def find_relevant_page(query) :
    query = query.lower()
    keyword_list = [0 for I in range(10000)]
    counterL = 0 
    for i in range(counter) :
        if keyword[i] in query :
            #print(keyword[i])
            keyword_list[counterL] = keyword[i]
            counterL += 1
    return list(set(keyword_list[:counterL]))

def is_title_imp(query) :
    query = query.lower()
    keyword_list = [0 for I in range(10000)]
    counterL = 0 
    for i in range(10088) :
        if title[i].lower() in query :
            #print(keyword[i])
            for j in range(len(data_categories[i])) :
                if 'births' in data_categories[i][j].lower() :
                    keyword_list[counterL] = title[i]
                    counterL += 1
                    print("Added .......",title[i],"\n")
                    break 
                    
                if 'deaths' in data_categories[i][j].lower() :
                    keyword_list[counterL] = title[i]
                    counterL += 1
                    print("Added .......",title[i],"\n")
                    
    
    return list(set(keyword_list[:counterL]))



def find_keyword(query) :
    num = 1  #That is title found
    List = is_title_imp(query) 
    if( not len(List)) :
        print("No title found .........................\n\n")
        List = find_relevant_page(query)
        num = 0 
    #### Removing the keyword "Who" from it 
    if "who" in List :
        List.remove("who")
    return List ,num
            
   

In [48]:
### Now it may happen that List from find_keyword contains some subset of another keyword of list , so we have to remove that .
"""
For example if List contain = ['Cheif Minister' , 'Minister' ,  'Chief Minister of Chhattishgarh'] , so it should be known to 
the system that user is searching for 'Chief Minister of Chhattishgarh' and to implement this it will just contain the largest 
set of all the intersecting keywords 
"""
def unique(keywords,num):
    keys = []
    for s in keywords:
        if not any([s in r for r in keywords if s != r]):
            keys.append(s)
    
    return keys,num

In [18]:
%%time
for i in range(10088) :
    ## Converting all cat of this title to lower 
    for j in range(len(data_categories[i])) :
        data_categories[i][j] = data_categories[i][j].lower()

Wall time: 5.14 s


In [50]:
"""
Here ->
1) Case 0 is when we had found a title in keyword 
2) Case 1 is when length of keyword is just 1 and if there is any title containing that match word 
3) Case 1 is searching for all the data_categories 

"""



def content_find(keywords,num) :
    if(num == 1) :
        print("Case 0\n")
        contentZero = [0 for i in range(10088)]
        contentLent = 0 
        for i in range(10088) :
            if keywords[0].lower() == title[i].lower() :
                contentZero[contentLent] = content[i].lower() 
                contentLent += 1
        if(contentLent) :
            return contentZero[:contentLent] , 1
    
    contents = {}
    content_final = [0 for i in range(1000)]
    final_counter =  0 
    
    
    list_of_title = [0 for i in range(100088)] 
    counterL = 0 
    done = 0 # Done is 1 if we have got titles matching keywords 
    ## First looking for any title matching [But only if number of keywords is 1]
    if(len(keywords) == 1) :
        print("Case 1")
        for i in range(10088) :
            if keywords[0].lower() in title[i].lower() :
                print(title[i].lower()," and index is ",i,"\n")
                done = 1
                contents[get_cosine(text_to_vector(keywords[0]),text_to_vector(title[i]))] = (content[i],title[i])
                    
        
        l = list(contents.items())
        l.sort(reverse=True)
        contents = dict(l)
       # print("Sorted and converted to dictionary again ")
        only5 = 0 
        for k in contents :
            only5+=1 
            content_final[final_counter] = contents[k][0]
            final_counter += 1
            #print("Content and title are  ..........",contents[k][0],"\n\n\n",contents[k][1])
            if(only5 == 5) :
                break 
        if(done == 1 ) :
            return content_final[:final_counter] ,1 
    
    
    ### The case we'll have to return list of answer is left 
    index_title = [0 for i in range(100088)] 
    
    if(done == 0) :
        print("Case 2 ")
        all_titles = [0 for i in range(10088)]
        for i in range(10088) : 
            done = 0 
            lenK = len(keywords)
            for k in range(lenK) :
                for z in range(len(data_categories[i])) :
                    if keywords[k] in data_categories[i][z].lower() :
                        done += 1 
                        break 
            if(done == lenK) :
                list_of_title[counterL] = title[i] 
                index_title[counterL] = i
                counterL += 1 
                print("Title added is ...................",title[i])
        if(counterL < 4) :
            print("Inside here")
            for i in range(4) :
                content_final[final_counter] = content[index_title[i]]
                print(content_final[final_counter])
                final_counter += 1 
            return content_final[:final_counter] , 1
            
            
        return list_of_title[:counterL],2


In [51]:
def all_preprocessing(query) :
    
    query = query.lower()
    
    #This sample keyword is to make sure that we aren't doing spelling correction if we found a title in query 
    sample_keyword , sample_num  = find_keyword(query.lower())
    if(sample_num == 0):
        ##Spelling correction 
        correct = correction(query) 
    else :
        print("Already corrected title")
    done = int(input()) 
    while(not done) :
        print("Input your query again .......................")
        query = input(query)
        #correct = correction(query)
        done = int(input())
    imp_keywords,num = find_keyword(query)
    imp_keywords,num = unique(imp_keywords,num)
    #### Removing the extra ' ' from front from all keywords 
    for i in range(len(imp_keywords)) :
        if(imp_keywords[i][0] == ' ') :
            #print(imp_keywords[i])
            imp_keywords[i] = imp_keywords[i][1:] 
    print("imp keyword is ................",imp_keywords)
    content_list , number = content_find(imp_keywords,num)
    
    return content_list , number 

    

#### Preprocessing Part is done. Let's see the use of ALBERT  

In [22]:
%%time

import torch
from transformers import AlbertTokenizer, AlbertForQuestionAnswering
tokenizer = AlbertTokenizer.from_pretrained('ahotrod/albert_xxlargev1_squad2_512')
model = AlbertForQuestionAnswering.from_pretrained('D:/albert')

Wall time: 33.5 s


In [23]:
def answer(question, text):
    input_dict = tokenizer.encode_plus(question, text, return_tensors='pt')
    input_ids = input_dict["input_ids"].tolist()
    start_scores, end_scores = model(**input_dict,return_dict=False)
    
    ##Return_dict is important 
    start = torch.argmax(start_scores)
    end = torch.argmax(end_scores)
    
    #print("Scores are " ,start_scores ," ", end_scores,"\n")
    
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    answer = ''.join(all_tokens[start: end + 1]).replace('▁', ' ').strip()
    answer = answer.replace('[SEP]', '')
    return answer if answer != '[CLS]' and len(answer) != 0 else 'could not find an answer'

In [24]:
pattern3 = re.compile(r"[\n]+")
pattern4 = re.compile(r"[=]+") 


In [27]:
def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [28]:
%%time 

import gensim
from gensim.models import Word2Vec
from pyemd import emd

model_wmd = gensim.models.KeyedVectors.load_word2vec_format('D:/GoogleNews-vectors-negative300.bin.gz', binary=True)  

Wall time: 43.7 s


In [52]:
def finding_answer(Con_list,query) :
    vector1 = text_to_vector(query)
    pattern3 = re.compile(r"[\n]+")
    pattern4 = re.compile(r"[=]+") 
    
    for i in range(len(Con_list)) :
        Con_list[i] = pattern3.sub("",Con_list[i])
        Con_list[i] = pattern4.sub("",Con_list[i])
    
    text = ""
    for i in range(len(Con_list)) :
        text += Con_list[i] 
        text += " "
        
    text = text.split(".")
    
    
    max_score = {}
    for i in range(0,len(text)-1,1) :
        context = text[i] + text[i+1]
        vector2 = text_to_vector(context)
        max_score[get_cosine(vector1,vector2)] = i 

    l = list(max_score.items())
    l.sort(reverse=True)
    max_score= dict(l)
    
    
    ans = "could not find an answer"
    done = 0 
    for k in max_score :
        index = max_score[k] 
        context = ""
        if(index != len(text)-1) :
            context = context = text[index] + text[index+1]
        else :
            context = context = text[index] + text[index-1]
        ans = answer(query,context)
        #print(ans,context)
        #print(done)
        done += 1 
        if(ans != "could not find an answer" ) :
            ans = ans.replace("[CLS]","")
            ans = ans.replace(query.lower(),"")

            print(ans) 
            print("And context is ....................\n")
            print(context)
            break 
        if(done == 20) :
            print("Sorry couldn't find a perfect answer !!!! :((")
            break 
    
    
    
    
        
        
    
    
    

In [53]:
def complete(query) :
    query = query.lower()
    print("Query is ..............",query)
    Con_list , num_type = all_preprocessing(query)
    print()
    #print("Done preprocessing")
    #print(Con_list ,"is the content")
    if(num_type == 1) :
        answer = finding_answer(Con_list,query)
        print(answer)
    
    else :
        print(Con_list)
    

In [56]:
%%time 
complete("When mahendra singh dhoni started playing")

Query is .............. when mahendra singh dhoni started playing
Added ....... Mahendra Singh Dhoni 

Already corrected title
1
Added ....... Mahendra Singh Dhoni 

imp keyword is ................ ['Mahendra Singh Dhoni']
Case 0


 within 1 year, dhoni moved from playing in ccl to the bihar ranji team
And context is ....................

 within 1 year, dhoni moved from playing in ccl to the bihar ranji team dhoni has credited deval sahay for instilling discipline in him
None
Wall time: 5.67 s


In [57]:
%%time 
complete("Current prime minister of India")

Query is .............. current prime minister of india
No title found .........................


Searching for .......     ['current', 'prime', 'minister', 'india'] 

Here is the list of similar words you may be looking for ...............

current      ['currencies', 'courant', 'current', 'currency', 'burrett'] 

prime      ['prime', 'crime', 'primetime', 'pride', 'prize', 'crimes', 'pre', 'prizes', 'prism', 'primates', 'primate', 'prince', 'pritam', 'purie'] 

minister      ['ministers', 'ministry', 'sister', 'winter', 'ministership', 'westminster', 'feminists', 'ministries', 'minsters', 'feminist', 'miniseries', 'inter', 'minister', 'ministerial', 'ministers1927', 'ministersall', 'mitter', 'master', 'mistri', 'mistry', 'mister'] 

india      ['hindi', 'india', 'indian', 'indies', 'indiaall', 'india1933', 'indians', 'india1982', 'scindia', 'india1959', 'sindhi', 'nadia', 'indira', 'indonesia', 'india1950', 'india1932', 'india2013', 'india17th', 'india1919', 'india1907', 'india1952'

In [58]:
%%time 
complete("Who was ramanujan")

Query is .............. who was ramanujan
Added ....... Ramanuja 

Already corrected title
1
Added ....... Ramanuja 

imp keyword is ................ ['Ramanuja']
Case 0


yadava prakasa
And context is ....................

 his philosophical foundations for devotionalism were influential to the bhakti movementramanuja's guru was yādava prakāśa, a scholar who was a part of the more ancient advaita vedānta monastic tradition
None
Wall time: 4.59 s


In [59]:
%%time 
complete("When is Independence day celebrated")

Query is .............. when is independence day celebrated
No title found .........................


Searching for .......     ['independence', 'day', 'celebrities'] 

Here is the list of similar words you may be looking for ...............

independence      ['independence', 'dependent', 'independent', 'independents'] 

day      ['day', 'daily', 'today', 'days', 'da', 'dayal', 'dairy', 'danny', 'datey', 'uday', 'udaya', 'ay', 'daisy', 'daya'] 

celebrated      ['elected', 'related', 'created', 'venerated', 'federated', 'celebrities', 'celebrity'] 

If the suggestion is correct then press continue otherwise you can research your query
1
No title found .........................


imp keyword is ................ ['independence', 'ay']
Case 2 
Title added is ................... Mujibur Rehman
Title added is ................... Birsa Munda
Title added is ................... Pherozeshah Mehta
Title added is ................... Hem Barua (Tyagbir)
Title added is ................... Bakht K

Title added is ................... Swadeshabhimani Ramakrishna Pillai
Title added is ................... A. P. Udhayabhanu
Title added is ................... Narayandas Malkani
Title added is ................... V. K. Krishna Menon
Title added is ................... Lokmanya Tilak
Title added is ................... K. B. Sahay
Title added is ................... Harekrushna Mahatab
Title added is ................... Munishwar Dutt Upadhyay
Title added is ................... Lalithambika Antharjanam
Title added is ................... Gouthu Latchanna
Title added is ................... Indo-Pakistani War of 1971
Title added is ................... Kushal Konwar
Title added is ................... Veer Surendra Sai
Title added is ................... Accamma Cherian
Title added is ................... Jawaharlal Nehru
Title added is ................... Subhash Chandra Bose
Title added is ................... Fakhruddin Ali Ahmed
Title added is ................... Mohammed Abdul Rahiman

['Mujib

In [60]:
complete("15th august")

Query is .............. 15th august
No title found .........................


Searching for .......     ['15th', 'august'] 

Here is the list of similar words you may be looking for ...............

15th      ['19th', '17th', '12th', '13th', '15th', '16th', '14th', '5th', '18th', '11th', '10th', '20185th', '20195th'] 

august      ['august', 'augustine'] 

If the suggestion is correct then press continue otherwise you can research your query
1
No title found .........................


imp keyword is ................ []
Case 2 
Title added is ................... Manushi Chhillar
Title added is ................... Lala Amarnath
Title added is ................... H.S.S. Lawrence
Title added is ................... Sarigama Viji
Title added is ................... Raza Murad
Title added is ................... Ramendra Sundar Tribedi
Title added is ................... Nataraja Guru
Title added is ................... Nanditha
Title added is ................... Sukhjit Singh (MVC)
Title added

Title added is ................... T.M. Krishna
Title added is ................... Anurag Basu
Title added is ................... Braj Kumar Nehru
Title added is ................... Encyclopædia Britannica
Title added is ................... Pachaiyappa Mudaliar
Title added is ................... Sasikrishnan
Title added is ................... Pritish Chakraborty
Title added is ................... Pannalal Patel
Title added is ................... Bajranglal Takhar
Title added is ................... Suryakantham (actress)
Title added is ................... Kalakshetra
Title added is ................... Hemanta Mukherjee
Title added is ................... D.R. Bendre
Title added is ................... Roop Singh
Title added is ................... Ananda Chandra Dutta
Title added is ................... Uma Bose
Title added is ................... Rajahnate of Cebu
Title added is ................... Hemchandra Kanungo
Title added is ................... Jainism
Title added is ................

Title added is ................... Ramana Reddy
Title added is ................... Member of the Order of the British Empire
Title added is ................... Culture of Mysore
Title added is ................... Kan Singh Parihar
Title added is ................... Divya Dutta
Title added is ................... Jitendra Mohan Hans
Title added is ................... Naseem Banu
Title added is ................... Nabakrishna Deb
Title added is ................... Surya Shekhar Ganguly
Title added is ................... Pramathesh Barua
Title added is ................... Allan Sealy
Title added is ................... Ramananda Chatterjee
Title added is ................... Kabi Samrat Upendra Bhanja
Title added is ................... Mani Madhava Chakyar
Title added is ................... Potana
Title added is ................... Basavanna
Title added is ................... Radhika Menon (naval officer)
Title added is ................... Sheila Dikshit
Title added is ................... Ar

Title added is ................... Kalyanasundara Mudaliar
Title added is ................... Birendranath Sasmal
Title added is ................... Anju Bobby George
Title added is ................... Ratish Nanda
Title added is ................... P.S.Krishnaprasad
Title added is ................... Asa Singh Mastana
Title added is ................... S. Regupathy
Title added is ................... Papon (Assamese singer)
Title added is ................... Lists of Indian people
Title added is ................... Ha Ma Nayak
Title added is ................... Arundhati Nag
Title added is ................... Deep Tyagi
Title added is ................... Anirban Lahiri
Title added is ................... List of Argentines
Title added is ................... Kodi Ramakrishna
Title added is ................... List of chief ministers of Puducherry
Title added is ................... Kochi (India)
Title added is ................... Maria Irudayam
Title added is ................... Pope John

Title added is ................... Zafar Iqbal (field hockey)
Title added is ................... Meera Jasmine
Title added is ................... Panjabi MC
Title added is ................... Nandamuri Taraka Rama Rao
Title added is ................... Agatti Airport
Title added is ................... Devapala (Paramara dynasty)
Title added is ................... Railway Minister of India
Title added is ................... Gagan Narang
Title added is ................... Raghavendra Swami
Title added is ................... A. Palanisamy
Title added is ................... Shubham Mavi
Title added is ................... Art and culture of Karnataka
Title added is ................... Ravindra Jain
Title added is ................... B. Nagi Reddy
Title added is ................... Arjun Munda
Title added is ................... Kamal Varadoor
Title added is ................... Akkineni Nageswara Rao
Title added is ................... Thanu Padmanabhan
Title added is ................... Palpu

Title added is ................... Pankaj Jha
Title added is ................... Arun Mukherjee
Title added is ................... Amar Shahid Bandhu Singh
Title added is ................... Gopal Swarup Pathak
Title added is ................... Nookala Chinna Satyanarayana
Title added is ................... T.K.Chand
Title added is ................... Dimapur district
Title added is ................... Sanjay Dutt
Title added is ................... Shivananda
Title added is ................... List of Baltic Russians
Title added is ................... P. K. Jayalakshmi
Title added is ................... Urdu
Title added is ................... Balwinder Sandhu
Title added is ................... Madathilparampil Mammen Thomas
Title added is ................... Salma (writer)
Title added is ................... Hemvati Nandan Bahuguna
Title added is ................... List of people from Dadra and Nagar Haveli
Title added is ................... Ramcharitmanas
Title added is .............

Title added is ................... List of Chuvashs
Title added is ................... Languages of Karnataka
Title added is ................... Alfredo Nobre da Costa
Title added is ................... Dasarath Deb
Title added is ................... Kedar Nath Sahoo
Title added is ................... Deepak Dev
Title added is ................... Ranabai
Title added is ................... Shiv Khera
Title added is ................... University of Alberta
Title added is ................... Saurabh Shukla
Title added is ................... Munger (Lok Sabha constituency)
Title added is ................... List of chief ministers from the Bharatiya Janata Party
Title added is ................... Kalanath Mishra
Title added is ................... Sindhu Vee
Title added is ................... Shrinivas Khale
Title added is ................... List of North Karnataka historical sites
Title added is ................... Guru Gobind Singh
Title added is ................... Amitav Ghosh
Title a

Title added is ................... Abdul Basith
Title added is ................... Lord Rama
Title added is ................... Saharanpur
Title added is ................... Revolutionary
Title added is ................... Sharad Kumar (athlete)
Title added is ................... Ashoka
Title added is ................... G. S. Bali
Title added is ................... Kamala Harris
Title added is ................... Aceh
Title added is ................... Rajya Sabha
Title added is ................... Vijaya Raje
Title added is ................... List of Nigeriens
Title added is ................... Aaj Tak
Title added is ................... Aamir Khan
Title added is ................... Shivarajkumar
Title added is ................... Kafeel Ahmed
Title added is ................... Meerut
Title added is ................... Annie Funk
Title added is ................... Mohammed Shami
Title added is ................... Shah Faesal
Title added is ................... Sridevi
Title added is .